In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import scale
from sklearn.cluster import AgglomerativeClustering, KMeans
from scipy.cluster import hierarchy

from scipy.stats import f
from scipy.spatial.distance import mahalanobis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

%matplotlib inline 
# авто вывод графиков (отпадает необходимость писать plt.show())

In [2]:
FEATURES = ["x1", "x2", "x3", "x4", "x5"] 

TRAIN_SAMPLES = {
    1 : [5,9,12,15,20,24,26,34],
    2 : [42, 44],
    3 : [2,5,11,22,25,29,31,35],
}
print(TRAIN_SAMPLES)

{1: [5, 9, 12, 15, 20, 24, 26, 34], 2: [42, 44], 3: [2, 5, 11, 22, 25, 29, 31, 35]}


In [3]:
data = pd.read_excel("C:\Games\Ekonometrika\data_for_lab2_part2_1.xlsx")
data.head()

,x1,x2,x3,x4,x5,x6
1,20.8,58.5,69,15626,0.1,NaN
2,12.4,93.0,180,23410,1.4,3.0
3,11.9,39.8,224,9855,0.7,NaN
4,13.6,55.0,116,4881,0.2,3.0
5,17.2,63.5,172,15466,5.3,1.0


In [4]:
def gen_train_data(data, features, train_samples=None):
    if train_samples:
        print('lol')
    else:
        #assert data.shape[1] == len(features) + 1
        train_data = data.dropna()
        cls_col = train_data.drop(columns=features).columns[0]
        train_data = train_data.rename({cls_col: "Class"}, axis=1)
    #train_data = train_data.astype({"Class": 'int32'}) 
    return train_data
        
train_data = gen_train_data(data, FEATURES)
train_data.head()

,x1,x2,x3,x4,x5,Class
2,12.4,93.0,180,23410,1.4,3.0
4,13.6,55.0,116,4881,0.2,3.0
5,17.2,63.5,172,15466,5.3,1.0
9,17.7,75.3,77,28919,0.2,1.0
11,11.7,51.4,69,7589,7.9,3.0


In [5]:
def scatter_matrix(samples):
    d = samples - samples.mean()
    print(d)
    res = np.zeros((d.shape[1], d.shape[1]))
    print(res)
    for _, row in d.iterrows():
        col = row.to_frame()
        print('help1',col)
        print('help2',col @ col.T)
        res += col @ col.T
    print(res)
    return res

def classes_scatter_matrix(samples, labels):
    A = np.zeros((samples.shape[1], samples.shape[1])) #создаем массив с нулями
    print(A)
    for cls in labels.unique(): #находит уникальные значения
        A += scatter_matrix(samples[labels == cls])
    return A

print(train_data.Class)
print('333',classes_scatter_matrix(train_data[FEATURES],train_data.Class))

2     3.0
4     3.0
5     1.0
9     1.0
11    3.0
12    1.0
15    1.0
20    1.0
22    3.0
24    1.0
25    3.0
26    1.0
29    3.0
31    3.0
34    1.0
35    3.0
42    2.0
44    2.0
Name: Class, dtype: float64
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
        x1    x2      x3        x4       x5
2  -0.6875  17.9  64.125   7491.75  -3.3125
4   0.5125 -20.1   0.125 -11037.25  -4.5125
11 -1.3875 -23.7 -46.875  -8329.25   3.1875
22 -1.2875  22.3  80.125    108.75  12.3875
25  0.6125  11.8  -8.875  18516.75  -2.2125
29  1.5125 -17.2 -22.875 -11455.25  -4.7125
31 -0.5875  -3.3 -11.875   4158.75   2.8875
35  1.3125  12.3 -53.875    545.75  -3.7125
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
help1             2
x1    -0.6875
x2    17.9000
x3    64.1250
x4  7491.7500
x5    -3.3125
help2              x1            x2             x3            x4            x5
x1     0.472656     -12.30625     -44.085938 

In [6]:
testdf = pd.DataFrame([[1, 1.5]], columns=['int', 'float'])
row = next(testdf.iterrows())[1]
print(testdf)
print(row)

   int  float
0    1    1.5
int      1.0
float    1.5
Name: 0, dtype: float64


In [17]:
cov = pd.DataFrame(
    classes_scatter_matrix(train_data[FEATURES], train_data.Class) / (train_data.shape[0] - train_data.Class.unique().size), 
    index=FEATURES, 
    columns=FEATURES
)
cov.head

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
        x1    x2      x3        x4       x5
2  -0.6875  17.9  64.125   7491.75  -3.3125
4   0.5125 -20.1   0.125 -11037.25  -4.5125
11 -1.3875 -23.7 -46.875  -8329.25   3.1875
22 -1.2875  22.3  80.125    108.75  12.3875
25  0.6125  11.8  -8.875  18516.75  -2.2125
29  1.5125 -17.2 -22.875 -11455.25  -4.7125
31 -0.5875  -3.3 -11.875   4158.75   2.8875
35  1.3125  12.3 -53.875    545.75  -3.7125
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
help1             2
x1    -0.6875
x2    17.9000
x3    64.1250
x4  7491.7500
x5    -3.3125
help2              x1            x2             x3            x4            x5
x1     0.472656     -12.30625     -44.085938 -5.150578e+03      2.277344
x2   -12.306250     320.41000    1147.837500  1.341023e+05    -59.293750
x3   -44.085938    1147.83750    4112.015625  4.804085e+05   -212.414063
x4 -5150.578125  134102.32500  480

<bound method NDFrame.head of             x1             x2            x3            x4             x5
x1    0.972583      -2.306000 -4.034167e+00 -3.415050e+02      -1.730917
x2   -2.306000     228.072333 -2.928500e+02  1.228396e+05     -32.641667
x3   -4.034167    -292.850000  2.122649e+04 -2.142470e+06    1807.107500
x4 -341.505000  122839.590000 -2.142470e+06  2.966591e+08 -194841.238333
x5   -1.730917     -32.641667  1.807107e+03 -1.948412e+05     179.418500>

In [8]:
lda = LinearDiscriminantAnalysis().fit(train_data[FEATURES], train_data.Class)

print(lda.classes_, lda.means_)

means = pd.DataFrame(lda.means_, index=lda.classes_, columns=FEATURES)
means

[1. 2. 3.] [[1.645000e+01 7.055000e+01 1.360000e+02 1.682150e+04 4.687500e+00]
 [1.660000e+01 1.140500e+02 5.675000e+02 5.978550e+04 1.216000e+02]
 [1.308750e+01 7.510000e+01 1.158750e+02 1.591825e+04 4.712500e+00]]


,x1,x2,x3,x4,x5
1.0,16.4500,70.55,136.000,16821.50,4.6875
2.0,16.6000,114.05,567.500,59785.50,121.6000
3.0,13.0875,75.10,115.875,15918.25,4.7125


In [9]:
def get_df_coef(lda, features):
    if lda.classes_.size == 2: #если 2 класса, то lda вернет разность коэффициентов функций
        return pd.DataFrame(np.concatenate([lda.intercept_, lda.coef_.ravel()]), index=["Intercept"] + features, columns=["Discriminant F"])
# если 3 и более классов, то lda вернет несколько функций (аналогично ППП Statistica)
    return pd.DataFrame(
        np.vstack([lda.intercept_, lda.coef_.T]),
        index=["Intercept"] + features,
        columns=lda.classes_
    )

In [10]:
df_coef = get_df_coef(lda, FEATURES) # применим эту функцию для наших параметров
df_coef

,1.0,2.0,3.0
Intercept,-11.455689,-423.228453,45.111224
x1,0.868085,8.154125,-2.906616
x2,0.055209,-0.557163,0.084082
x3,0.012508,-0.106458,0.014107
x4,-0.000225,0.002340,-0.000360
x5,-0.424308,4.169994,-0.618191


In [11]:
def calc_mahalanobis_square_dist(centers, samples, cov):
    # создаем матрицу расстояний между классами и объекnами
    res = pd.DataFrame(index=samples.index, columns=centers.index)
    for i in centers.index: # проходим по всем центрам классов
        for j in samples.index: # проходим по всем объектам
            # считаем расстояние между текущим классом и объектом
            res[i][j] = mahalanobis(centers.loc[i], samples.loc[j], np.linalg.inv(cov)) ** 2 
    return res

In [12]:
# выведем квадраты расстояний Махаланобиса между классами
center_distance = calc_mahalanobis_square_dist(means, means, cov)
center_distance

,1.0,2.0,3.0
1.0,0,570.458,12.9088
2.0,570.458,0,637.554
3.0,12.9088,637.554,0


In [13]:
lda.priors_ 

array([0.44444444, 0.11111111, 0.44444444])

In [14]:
def LDA_predict(lda, X):
    return pd.DataFrame(
        lda.predict(X),
        columns=["Class"],
        index=X.index
    )

In [15]:
lda_predict_train = LDA_predict(lda, train_data[FEATURES])
# сравниваем полученные классы с теми, которые были заданы в начале, и смотрим долю совпадения
(lda_predict_train.Class == train_data.Class).value_counts(normalize=True)

True    1.0
Name: Class, dtype: float64

In [16]:
lda_predict = LDA_predict(lda, data[FEATURES])
lda_predict.head()
print(lda_predict)

    Class
1     1.0
2     3.0
3     3.0
4     3.0
5     1.0
6     3.0
7     1.0
8     1.0
9     1.0
10    1.0
11    3.0
12    1.0
13    1.0
14    2.0
15    1.0
16    1.0
17    1.0
18    1.0
19    1.0
20    1.0
21    3.0
22    3.0
23    2.0
24    1.0
25    3.0
26    1.0
27    3.0
28    1.0
29    3.0
30    1.0
31    3.0
32    3.0
33    3.0
34    1.0
35    3.0
36    1.0
37    1.0
38    2.0
39    1.0
40    1.0
41    1.0
42    2.0
43    1.0
44    2.0
45    3.0
46    3.0
47    3.0
